# Spaceship Titanic - Kaggle competition

https://www.kaggle.com/competitions/spaceship-titanic

This notebook serves as an introduction for the class about MLFlow and Optuna.
The students are introduced to the concept of Hyperparameter Tuning and experiment tracking with MLFlow.

In [1]:
# pip install pandas

We load the train data. The PassengerId column is used as the index of the dataframe

In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv("Data/train.csv", index_col= "PassengerId")
train

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Name          8493 non-null   object 
 12  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(6)
memory usage: 891.4+ KB


## Preprocessing Pipeline

We identified null values in all columns. We will clean these by type.

In [4]:
train.isna().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [ ]:
# pip install scikit-learn

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

train = pd.read_csv("Data/train.csv", index_col= "PassengerId")

# Step 1: Define transformers for different column types
numerical_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean"))]
)

categorical_cols = ['HomePlanet', 'Destination', 'VIP', 'CryoSleep']
categorical_transformer = Pipeline(
    steps=[
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Step 2: Create a ColumnTransformer that applies the transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop' 
)

# Step 3: Assemble the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# Fit and transform the DataFrame
X_preprocessed = preprocessing_pipeline.fit_transform(train)

preprocessing_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['HomePlanet', 'Destination',
                                                   'VIP', 'CryoSleep'])]))])

In [6]:
# Converting back to Pandas DataFrame
onehot_encoder_feature_names = list(preprocessing_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['encoder'].get_feature_names_out())
column_order =  numerical_cols + onehot_encoder_feature_names

# Show the cleaned DataFrame
X_preprocessed = pd.DataFrame(X_preprocessed, columns=column_order, index=train.index)
y = train['Transported']

X_preprocessed.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_nan,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_nan,VIP_False,VIP_True,VIP_nan,CryoSleep_False,CryoSleep_True,CryoSleep_nan
PassengerId,,,,,,,,,,,,,,,,,,,,
0001_01,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
0002_01,24.0,109.0,9.0,25.0,549.0,44.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
0004_01,16.0,303.0,70.0,151.0,565.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


## Hyperparameters tuning of a Decision Tree Classifier

We extend the pipeline with a decision tree classifier to predict the Transported variable. 

In [7]:
# pip install optuna

In order to have a local tracking server run the following command in your terminal:

In [8]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

model = DecisionTreeClassifier(criterion='entropy', random_state= 42)

def objective(trial):

    params = {
        # trial parameters to optimize
        'max_depth' : trial.suggest_int('max_depth', 3, 40, log=True),
        'min_samples_split' : trial.suggest_float('min_samples_split', 1e-6, 1e-3, log=True),
        'min_samples_leaf' : trial.suggest_float('min_samples_leaf', 1e-6, 1e-3, log=True)
    }

    model.set_params(**params)

    cv_score = cross_val_score(model, X_preprocessed, y, cv=5, scoring='accuracy', n_jobs=-1).mean()

    return cv_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("--------------------------------------")
print("best_params =", study.best_params, "with cross_validation_score =", study.best_value)

/Users/levkiwi/Dev/64-61.1-Data-Science/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-05-02 18:09:36,371] A new study created in memory with name: no-name-b8bba99d-beab-48cf-b2d2-ad143c6d7d9b
[I 2024-05-02 18:09:37,388] Trial 0 finished with value: 0.73944689982934 and parameters: {'max_depth': 23, 'min_samples_split': 3.470724554763953e-06, 'min_samples_leaf': 4.591675158461207e-06}. Best is trial 0 with value: 0.73944689982934.
[I 2024-05-02 18:09:37,912] Trial 1 finished with value: 0.7631453601827962 and parameters: {'max_depth': 5, 'min_samples_split': 0.0005636073814629572, 'min_samples_leaf': 0.00017643140061191306}. Best is trial 1 with value: 0.7631453601827962.
[I 2024-05-02 18:09:38,461] Trial 2 finished with value: 0.7630308147679545 and parameters: {'max_depth': 14, 'min_sam

--------------------------------------
best_params = {'max_depth': 8, 'min_samples_split': 5.463427733938847e-06, 'min_samples_leaf': 0.000257282926445536} with cross_validation_score = 0.7836210644451959


### Using MLFlow for evaluation

The best model is saved in MLFlow for later use. First we need to start a local tracking server. After installing mlflow, execute the following command in the terminal:

mlflow server --host 127.0.0.1 --port 8080

In [ ]:
# pip install mlflow
# pip install setuptools

In [9]:
import mlflow
from mlflow.models import infer_signature

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("Spaceship_Titanic")

2024/05/02 18:13:18 INFO mlflow.tracking.fluent: Experiment with name 'Spaceship_Titanic' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/953378837648444733', creation_time=1714666398584, experiment_id='953378837648444733', last_update_time=1714666398584, lifecycle_stage='active', name='Spaceship_Titanic', tags={}>

In [10]:
train = pd.read_csv("Data/train.csv", index_col= "PassengerId")

X = train.drop(['Transported'], axis = 1)
y = train['Transported']

# Start an MLflow run
with mlflow.start_run():
    # Fit the model with the best hyperparameters from the study
    model = DecisionTreeClassifier(criterion= 'entropy', random_state= 42)
    model.set_params(**study.best_params)

    model_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])

    model_pipeline.fit(X, y)
    
    # Log the hyperparameters
    mlflow.log_params(study.best_params)

    # Log the loss metric
    mlflow.log_metric("accuracy", study.best_value)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Simple Decision Tree Classifier")

    # Infer the model signature
    signature = infer_signature(X, model_pipeline.predict(X))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model_pipeline,
        signature=signature,
        input_example=X,
        registered_model_name="decision_tree_model",
        artifact_path="decision_tree_model"
    )

Registered model 'decision_tree_model' already exists. Creating a new version of this model...
2024/05/02 18:17:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: decision_tree_model, version 4
Created version '4' of model 'decision_tree_model'.


## Hyperparameters tuning of a XGBoost Classifier

We extend the pipeline with a XGBoost classifier to predict the Transported variable. Since it uses some regularization, we need to adapt the pipeline and add a StandardScaler after the ColumnTransformer

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

train = pd.read_csv("Data/train.csv", index_col= "PassengerId")

# Step 1: Define transformers for different column types
numerical_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
numeric_transformer = Pipeline(
    steps=[
        ("imputer", IterativeImputer(random_state=0))]
)

categorical_cols = ['HomePlanet', 'Destination', 'VIP', 'CryoSleep']
categorical_transformer = Pipeline(
    steps=[
        ('encoder', OneHotEncoder())
])

# Step 2: Create a ColumnTransformer that applies the transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop' 
)

# Step 3: Assemble the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler())
])

# Fit and transform the DataFrame
X_preprocessed = preprocessing_pipeline.fit_transform(train)

preprocessing_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   IterativeImputer(random_state=0))]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['HomePlanet', 'Destination',
                                                   'VIP', 'CryoSleep'])])),
                ('scaler', StandardScaler())])

In [ ]:
# pip install xgboost

In [12]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

model = XGBClassifier(objective='binary:logistic', random_state=42)

def objective(trial):
    params = {
        # trial parameters to optimize for XGBoost
        'eta': trial.suggest_float('eta', 0.01, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 50, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'alpha': trial.suggest_float('alpha', 1e-6, 100, log=True),
        'lambda': trial.suggest_float('lambda', 1e-6, 100, log=True)
    }

    model.set_params(**params)

    cv_score = cross_val_score(model, X_preprocessed, y, cv=5, scoring='accuracy', n_jobs=-1).mean()

    return cv_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

print("--------------------------------------")
print("best_params =", study.best_params, "with cross_validation_score =", study.best_value)

[I 2024-05-02 18:23:27,766] A new study created in memory with name: no-name-3ac708fe-a9bf-441a-99fa-8687c5a6e2a3
[I 2024-05-02 18:23:29,000] Trial 0 finished with value: 0.7768340997266394 and parameters: {'eta': 0.07543338672921536, 'n_estimators': 287, 'max_depth': 19, 'min_child_weight': 4, 'subsample': 0.8828590626132958, 'colsample_bytree': 0.8516207851756938, 'gamma': 0.20422421273869307, 'alpha': 0.5331902981944754, 'lambda': 0.15859087440552067}. Best is trial 0 with value: 0.7768340997266394.
[I 2024-05-02 18:23:29,769] Trial 1 finished with value: 0.7731514414789394 and parameters: {'eta': 0.17328644029851312, 'n_estimators': 313, 'max_depth': 10, 'min_child_weight': 6, 'subsample': 0.6282470549700192, 'colsample_bytree': 0.6162553000096547, 'gamma': 0.3310935721539312, 'alpha': 0.004750520075503199, 'lambda': 3.4580760568746343e-06}. Best is trial 0 with value: 0.7768340997266394.
[I 2024-05-02 18:23:30,377] Trial 2 finished with value: 0.7757982941931232 and parameters: {'

--------------------------------------
best_params = {'eta': 0.02265086669742111, 'n_estimators': 184, 'max_depth': 7, 'min_child_weight': 2, 'subsample': 0.7261753375725644, 'colsample_bytree': 0.9593115536275667, 'gamma': 0.35949618555363216, 'alpha': 3.598954143184485, 'lambda': 5.415102843248525e-06} with cross_validation_score = 0.8012214868934503


In [13]:
train = pd.read_csv("Data/train.csv", index_col= "PassengerId")

X = train.drop(['Transported'], axis = 1)
y = train['Transported']

# Start an MLflow run
with mlflow.start_run():
    # Fit the model with the best hyperparameters from the study
    model = XGBClassifier(objective='binary:logistic', random_state=42)
    model.set_params(**study.best_params)

    model_pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])

    model_pipeline.fit(X, y)
    
    # Log the hyperparameters
    mlflow.log_params(study.best_params)

    # Log the loss metric
    mlflow.log_metric("accuracy", study.best_value)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "XGBoost Classifier")

    # Infer the model signature
    signature = infer_signature(X, model_pipeline.predict(X))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model_pipeline,
        signature=signature,
        input_example=X,
        registered_model_name="XGBoost_model",
        artifact_path="XGBoost_model"
    )

Registered model 'XGBoost_model' already exists. Creating a new version of this model...
2024/05/02 18:25:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoost_model, version 3
Created version '3' of model 'XGBoost_model'.


## Random Forest

We extend the pipeline with a RandomForest classifier to predict the Transported variable. 

In [14]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=42, n_jobs=-1)

def objective(trial):
    
    params = {
        # trial parameters to optimize for RandomForestClassifier
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 50, log=True),
        'min_samples_split': trial.suggest_float('min_samples_split', 1e-6, 1e-3, log=True),
        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 1e-6, 1e-3, log=True),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0)
    }

    model.set_params(**params)

    cv_score = cross_val_score(model, X_preprocessed, y, cv=5, scoring='accuracy', n_jobs=-1).mean()

    return cv_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("--------------------------------------")
print("best_params =", study.best_params, "with cross_validation_score =", study.best_value)

[I 2024-05-02 18:26:14,378] A new study created in memory with name: no-name-b844a5a1-64a9-4bc5-8dd8-52da321312f5
[I 2024-05-02 18:26:22,544] Trial 0 finished with value: 0.7820096863996675 and parameters: {'n_estimators': 728, 'max_depth': 30, 'min_samples_split': 3.0046510300421914e-05, 'min_samples_leaf': 6.517900456035476e-05, 'max_features': 0.964128227719162}. Best is trial 0 with value: 0.7820096863996675.
[I 2024-05-02 18:26:26,248] Trial 1 finished with value: 0.7820104143023615 and parameters: {'n_estimators': 355, 'max_depth': 24, 'min_samples_split': 1.98140946865391e-05, 'min_samples_leaf': 3.814257537914162e-05, 'max_features': 0.8686215859352887}. Best is trial 1 with value: 0.7820104143023615.
[I 2024-05-02 18:26:28,111] Trial 2 finished with value: 0.7878773100157426 and parameters: {'n_estimators': 193, 'max_depth': 17, 'min_samples_split': 2.5971644157451432e-05, 'min_samples_leaf': 4.050142862056506e-06, 'max_features': 0.8150102780685032}. Best is trial 2 with valu

--------------------------------------
best_params = {'n_estimators': 487, 'max_depth': 8, 'min_samples_split': 0.0006588264647761022, 'min_samples_leaf': 1.634706193394958e-06, 'max_features': 0.6584025184692753} with cross_validation_score = 0.799610836750616


In [15]:
train = pd.read_csv("Data/train.csv", index_col= "PassengerId")

X = train.drop(['Transported'], axis = 1)
y = train['Transported']

# Start an MLflow run
with mlflow.start_run():
    # Fit the model with the best hyperparameters from the study
    model = RandomForestClassifier(random_state=42, n_jobs=-1)
    model.set_params(**study.best_params)

    model_pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])  

    model_pipeline.fit(X, y)
    
    # Log the hyperparameters
    mlflow.log_params(study.best_params)

    # Log the loss metric
    mlflow.log_metric("accuracy", study.best_value)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Random Forest Classifier")

    # Infer the model signature
    signature = infer_signature(X, model_pipeline.predict(X))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model_pipeline,
        signature=signature,
        input_example=X,
        registered_model_name="random_forest_model",
        artifact_path="random_forest_model"
    )

Registered model 'random_forest_model' already exists. Creating a new version of this model...
2024/05/02 18:31:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 2
Created version '2' of model 'random_forest_model'.
